In [ ]:
import numpy as np
import pandas as pd
import scipy
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from statsmodels.tsa.arima_model import ARIMA
!pip install arch
from arch import arch_model
import seaborn as sns
import yfinance
import warnings
warnings.filterwarnings("ignore")
sns.set()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 916.4/916.4 kB 9.0 MB/s eta 0:00:00


In [ ]:
pip install yfinance

In [ ]:
import yfinance as yf

In [ ]:
tickers = ["^GSPC", "^FTSE", "^N225", "^GDAXI"]
raw_data = yf.download(tickers=tickers, start="1994-01-07", end="2018-01-29", interval="1d", group_by='ticker', auto_adjust=True, threads=True)

[*********************100%%**********************]  4 of 4 completed


In [ ]:
df_comp = raw_data.copy()
df_comp['spx'] = df_comp['^GSPC'].Close[:]
df_comp['dax'] = df_comp['^GDAXI'].Close[:]
df_comp['ftse'] = df_comp['^FTSE'].Close[:]
df_comp['nikkei'] = df_comp['^N225'].Close[:]
df_comp = df_comp.iloc[1:]
del df_comp['^N225']
del df_comp['^GSPC']
del df_comp['^GDAXI']
del df_comp['^FTSE']
df_comp=df_comp.asfreq('b')
df_comp=df_comp.fillna(method='ffill')

In [ ]:
df_comp['ret_spx'] = df_comp.spx.pct_change(1)*100
df_comp['ret_ftse'] = df_comp.ftse.pct_change(1)*100
df_comp['ret_dax'] = df_comp.dax.pct_change(1)*100
df_comp['ret_nikkei'] = df_comp.nikkei.pct_change(1)*100

In [ ]:
size = int(len(df_comp)*0.8)
df, df_test = df_comp.iloc[:size], df_comp.iloc[size:]

In [ ]:
!pip install pmdarima
from pmdarima.arima import auto_arima
model_auto = auto_arima(df.ret_ftse[1:])
model_auto

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.0 MB/s eta 0:00:00


ARIMA(order=(4, 0, 5), scoring_args={}, suppress_warnings=True,
      with_intercept=False)

In [ ]:
model_auto.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 5019
Model:               SARIMAX(4, 0, 5)   Log Likelihood               -7882.776
Date:                Fri, 22 Sep 2023   AIC                          15785.552
Time:                        21:34:49   BIC                          15850.762
Sample:                    01-11-1994   HQIC                         15808.403
                         - 04-05-2013                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.0120      0.082      0.147      0.883      -0.148       0.172
ar.L2         -0.6543      0.077     -8.457      0.000      -0.806      -0.503
ar.L3         -0.1628      0.071     -2.290      0.022      -0.302      -0.023
ar.L4          0.2014      0.074      2.711      0.007       0.056       0.347
ma.L1         -0.0357      0.081     -0.440      0.660      -0.195       0.123
ma.L2          0.6068      0.078      7.769      0.000       0.454       0.760
ma.L3          0.0621      0.068      0.908      0.364      -0.072       0.196
ma.L4         -0.1933      0.073     -2.649      0.008      -0.336      -0.050
ma.L5         -0.1053      0.010    -11.067      0.000      -0.124      -0.087
sigma2         1.3543      0.015     91.437      0.000       1.325       1.383
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):              6354.62
Prob(Q):                              0.96   Prob(JB):                         0.00
Heteroskedasticity (H):               1.99   Skew:                            -0.20
Prob(H) (two-sided):                  0.00   Kurtosis:                         8.50
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [ ]:
model_auto = auto_arima(df_comp.ret_ftse[1:], exogenous = df_comp[['ret_spx', 'ret_dax', 'ret_nikkei']][1:], m = 5, max_order = None, max_p = 7, max_q = 7, max_d = 2, max_P = 4, max_Q = 4, max_D = 2, maxiter = 50, alpha = 0.05, n_jobs = -1, trend = 'ct', information_criterion = 'oob', out_of_sample = int(len(df_comp)*0.2))

In [ ]:
model_auto.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                      
===========================================================================================
Dep. Variable:                                   y   No. Observations:                 6274
Model:             SARIMAX(0, 0, 4)x(0, 0, [1], 5)   Log Likelihood               -9560.337
Date:                             Fri, 22 Sep 2023   AIC                          19136.673
Time:                                     21:52:58   BIC                          19190.627
Sample:                                 01-11-1994   HQIC                         19155.368
                                      - 01-26-2018                                         
Covariance Type:                               opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0246      0.028      0.884      0.377      -0.030       0.079
drift      -2.025e-06   7.91e-06     -0.256      0.798   -1.75e-05    1.35e-05
ma.L1         -0.0201      0.008     -2.496      0.013      -0.036      -0.004
ma.L2         -0.0496      0.007     -6.925      0.000      -0.064      -0.036
ma.L3         -0.0694      0.007     -9.724      0.000      -0.083      -0.055
ma.L4          0.0333      0.008      4.385      0.000       0.018       0.048
ma.S.L5       -0.0513      0.007     -7.261      0.000      -0.065      -0.037
sigma2         1.2386      0.012    104.270      0.000       1.215       1.262
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):              8783.36
Prob(Q):                              0.98   Prob(JB):                         0.00
Heteroskedasticity (H):               0.86   Skew:                            -0.17
Prob(H) (two-sided):                  0.00   Kurtosis:                         8.79
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""